In [3]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
from scipy import stats
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv
import statistics

In [4]:
rlist = []
records = os.path.normpath('mit-bih-dataframes/subject_list.csv')
with open(records) as rfile:
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [5]:
subject_rr_rhythms = []
for x in tqdm(rlist):
    subject_rr_rhythms.append(pd.read_csv(os.path.normpath('mit-bih-extracted/'+x+'_rr_rhythms.csv'), index_col=0))

100%|███████████████████████████████████████████| 23/23 [00:01<00:00, 12.39it/s]


In [6]:
def subset_subject(subjdf, interval_length = 25):
    if (len(subjdf) % interval_length != 0):
        slices = (len(subjdf) // interval_length) + 1
    else:
        slices = int(len(subjdf) / interval_length)

    subsetdf = []
    for x in tqdm(range(slices)):
        subsetdf.append(subjdf.iloc[interval_length*x:interval_length*x+interval_length].reset_index())
    return subsetdf

In [7]:
for subject in subject_rr_rhythms:
    drmean_column = []
    for index, value in subject["rmean"].items():
        if index==0:
            drmean_column.append(0)
        else:
            drmean_column.append(value-subject['rmean'][index-1])
    subject['drmean'] = drmean_column

In [8]:
subject_rr_rhythms[0].head()

,rr_int,rr_int_seconds,rhythms,rmean,rmean_seconds,drmean
0,139,0.556,N,139.000000,0.556000,0.000000
1,158,0.632,N,143.750000,0.575000,4.750000
2,226,0.904,N,164.312500,0.657250,20.562500
3,145,0.580,N,159.484375,0.637938,-4.828125
4,144,0.576,N,155.613281,0.622453,-3.871094


In [9]:
for x in tqdm(range(len(subject_rr_rhythms))):
    subject_rr_rhythms[x].to_csv('mit-bih-extracted/'+rlist[x]+'_rr_rhythms.csv')

100%|███████████████████████████████████████████| 23/23 [00:15<00:00,  1.49it/s]


In [26]:
drmean_variation = []
for subject in subject_rr_rhythms:
    drmeans = []
    subsets = subset_subject(subject)
    for subset in subsets:
        drmeans.append((subset['drmean'].var(), subset['rhythms'].mode().iloc[0]))
    drmean_variation.append(drmeans)

100%|█████████████████████████████████████| 2379/2379 [00:00<00:00, 2470.57it/s]


In [31]:
drmean_variation[0]

[(141.99119231512148, 'N'),
 (189.86223475631903, 'N'),
 (246.19573187106934, 'N'),
 (155.3235503811225, 'N'),
 (171.45972462224515, 'N'),
 (315.21845502678065, 'N'),
 (166.47298203033105, 'N'),
 (256.61305541174295, 'N'),
 (270.2026247248466, 'N'),
 (311.50424997048856, 'N'),
 (598.42826143211, 'N'),
 (65.07535088582812, 'N'),
 (250.75613903282806, 'N'),
 (174.2426296551424, 'N'),
 (163.2205660959499, 'N'),
 (455.63818733511283, 'N'),
 (263.55681856145384, 'N'),
 (234.81628591604354, 'N'),
 (54.660297576808425, 'N'),
 (138.79824480246788, 'N'),
 (57.228914003295955, 'N'),
 (67.79535939943717, 'A'),
 (39.8427620124586, 'A'),
 (110.58061209662681, 'A'),
 (152.24877132296558, 'N'),
 (138.07247570302675, 'N'),
 (154.88948055830375, 'N'),
 (83.7971927581019, 'A'),
 (77.92960348012828, 'A'),
 (278.1773570310506, 'A'),
 (33.22674405630068, 'A'),
 (14.47114489036052, 'A'),
 (66.08709370699894, 'A'),
 (88.5149301707364, 'N'),
 (26.476165537269285, 'N'),
 (16.850454834422166, 'N'),
 (1.79159558

In [48]:
drmean_dfs = []
for idx, subject in enumerate(drmean_variation):
    data = {
        "variance": [],
        "rhythm": []
    }
    for variation, rhythm in subject:
        data["variance"].append(variation)
        data["rhythm"].append(rhythm)
    df = pd.DataFrame(data)
    df.to_csv('mit-bih-extracted/'+rlist[idx]+'_drmean_variance.csv')